In [1]:
pip install simplejson

Note: you may need to restart the kernel to use updated packages.


>>> csv 파일 데이터 읽어온 후 "~구 코인세탁소이름" 형식으로 문자열 저장 

In [ ]:
import pandas as pd
import numpy as np

import re

#-- 1. 데이터 읽기 --#
data = pd.read_csv("laundry_data.csv", encoding="CP949")
#landry_name = list("서울특별시 " + data["사업장명"])
laundry_address = list(data["address"])
laundry_name = list(data["name"])

#-- 2. ~구 이름으로 문자열 나누기 --#
# 정규 표현식
p = re.compile('([가-힣]+(시|군|구|동|가))')
re = p.findall(laundry_address[0])
# print(re[1][0])


#-- 3. "~구 코인세탁소이름" 형식으로 데이터 저장 --# 
params = []

for index, ad in enumerate(laundry_address):
    re = p.findall(ad)
    params.append(re[0][0] + " "+f"{laundry_name[index]}")
    
# print(params)

>>> 키워드 검색 결과 DataFrame으로 만들기

In [242]:
# 키워드로 장소 검색하기

import simplejson,requests
import sys

import pandas as pd
import numpy as np

import re


url = "https://dapi.kakao.com/v2/local/search/keyword.json?"
apikey = "e10fc0ca482b5375d98fe727a94ba06b"

data = pd.read_excel("빨래방업.xlsx")
#landry_name = list("서울특별시 " + data["사업장명"])
laundry_address = list(data["소재지전체주소"])
laundry_name = list(data["사업장명"])

# 정규 표현식
p = re.compile('([가-힣]+(시|군|구|동))')
re = p.findall(laundry_address[0])
#print(re[2][0])


#-- 정규식 표현을 이용한 키워드 검색 --#
# for i,name in enumerate(laundry_name):
#     re = p.findall(laundry_address[i])
#     query = re[1][0] + name
#     print(query)
#     r = requests.get(url, params = {'query':query}, headers={'Authorization':'KakaoAK' + apikey})
#     print(r.json())
    

#-- DB --#
laundry_name = []
laundry_address = []
laundry_road = []
laundry_tel = []
laundry_lat = []
laundry_lng = []
laundry_url = []
washer_cnt = []
dryer_cnt = []

In [267]:
#-- 단일 쿼리 Test --#
query = "강동구 셀프빨래방"
r = requests.get( url, params = {'query':query}, headers={'Authorization' : 'KakaoAK ' + apikey } )

# laundry_address.append(r.json()['documents'][0]['address_name'])
# laundry_road.append(r.json()['documents'][0]['road_address_name'])
# laundry_url.append(r.json()['documents'][0]['id'])
# laundry_tel.append(r.json()['documents'][0]['phone'])
# laundry_name.append(r.json()['documents'][0]['place_name'])
# laundry_lat.append(r.json()['documents'][0]['x'])
# laundry_lng.append(r.json()['documents'][0]['y'])

for index in range(len(r.json()['documents'])):
    laundry_address.append(r.json()['documents'][index]['address_name'])
    laundry_road.append(r.json()['documents'][index]['road_address_name'])
    laundry_url.append(r.json()['documents'][index]['id'])
    laundry_tel.append(r.json()['documents'][index]['phone'])
    laundry_name.append(r.json()['documents'][index]['place_name'])
    laundry_lat.append(r.json()['documents'][index]['x'])
    laundry_lng.append(r.json()['documents'][index]['y'])

In [268]:
df = pd.DataFrame({'name':laundry_name, 'address':laundry_address, 'road':laundry_road, 'tel':laundry_tel,
                   'lat':laundry_lat, 'lng':laundry_lng, 'url':laundry_url})

>>> 세탁기 수, 건조기 수 더미 데이터 만들기

In [279]:
import random

for i in range(len(df)):
    washer_cnt.append(random.randrange(3,7))
    dryer_cnt.append(random.randrange(3,7))
    
df2 = pd.DataFrame({'washer_cnt':washer_cnt, 'dryer_cnt':dryer_cnt})

In [273]:
laundry_data = []
laundry_data = pd.concat([df,df2],axis=1) # 여기서 부터

In [276]:
laundry_data.to_csv('landry_data.csv', index=False, encoding="utf-8")

In [ ]:
df2.head(10)

>>> 카카오 API Test

In [172]:
# 주소 검색
import simplejson,requests
import sys

import pandas as pd
import numpy as np

import re
p = re.compile('서울특별시+')

url = "https://dapi.kakao.com/v2/local/search/address.json?"
apikey = "e10fc0ca482b5375d98fe727a94ba06b"

data = pd.read_excel("빨래방업.xlsx")
landry_name = list(data["소재지전체주소"])

x = []
y = []
link = []

#-- 단일 쿼리 Test --#
# query = "서울 마포구 환일길 60"
 
# r = requests.get( url, params = {'query':query}, headers={'Authorization' : 'KakaoAK ' + apikey } )
# r.json()
# x = r.json()['documents'][0]['x']
# y = r.json()['documents'][0]['y']

#-- DB에서 코인 세탁소 주소 가져오기 --#
for i,name in enumerate(landry_name):
    query = name
    r = requests.get(url, params = {'query':query}, headers={'Authorization': 'KakaoAK '+ apikey})
    x.append(r.json()['documents'][0]['x'])
    y.append(r.json()['documents'][0]['y'])
#     link.append(r.json()['documents'][0]['y'])
    
# # -- csv 파일로 변환 --#    
# new_pos = pd.DataFrame({'lat':x, 'lng':y})
# # #print(new_pos)
# new_data = pd.concat([data,new_pos],axis=1)

# new_data.to_csv('landry_data.csv', index=False, encoding="utf-8")    


{'documents': [{'address': {'address_name': '서울 마포구 아현동 751-3',
    'b_code': '1144010100',
    'h_code': '1144056500',
    'main_address_no': '751',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '마포구',
    'region_3depth_h_name': '공덕동',
    'region_3depth_name': '아현동',
    'sub_address_no': '3',
    'x': '126.961303685622',
    'y': '37.552976063588'},
   'address_name': '서울 마포구 환일길 60',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 마포구 환일길 60',
    'building_name': '',
    'main_building_no': '60',
    'region_1depth_name': '서울',
    'region_2depth_name': '마포구',
    'region_3depth_name': '아현동',
    'road_name': '환일길',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.961303685622',
    'y': '37.552976063588',
    'zone_no': '04203'},
   'x': '126.961303685622',
   'y': '37.552976063588'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [ ]:
# 주소 좌표 변환 TM -> WGS84
import simplejson,requests
import sys
import json

import pandas as pd
import numpy as np
from decimal import Decimal

import re
p = re.compile('서울특별시+')

path = "./sample.json"

url = "https://dapi.kakao.com/v2/local/geo/transcoord.json?"
apikey = "e10fc0ca482b5375d98fe727a94ba06b"

data = pd.read_excel("빨래방업.xlsx")
landry_pos = data.iloc[:58,8:10]
x = list(landry_pos['좌표정보(x)'])
y = list(landry_pos['좌표정보(y)'])
result_x = []
result_y = []


# query = "서울특별시 서대문구 북아현동 220-1번지 101호,102호"
# x=160710.37729270622&y=-4388.879299157299&input_coord=WTM&output_coord=WGS84"

#r = requests.get( url, params = {'query':query}, headers={'Authorization' : 'KakaoAK ' + apikey } )
#url = url + 'x=195784.6746&y=450601.9742&input_coord=TM&output_coord=WGS84'
#url = url + 'x=' + str(x[0]) + '&y=' + str(y[0]) + '&input_coord=TM&output_coord=WGS84'

for i in range(0,len(landry_pos)):
    url_pos = url + 'x=' + str(x[i]) + '&y=' + str(y[i]) + '&input_coord=TM&output_coord=WGS84'
    r = requests.get( url_pos, headers={'Authorization' : 'KakaoAK ' + apikey } )
    js = simplejson.JSONEncoder().encode(r.json())
    result_x.append((r.json()['documents'][0]['x']))
    result_y.append((r.json()['documents'][0]['y']))
    


# with open('pos.json','w', encoding="utf-8") as make_file:
#     json.dumps(data_pos, ensure_ascii=False, indent="\t")
    
# with open('test.json', 'w', encoding='utf-8') as make_file:
#     json.dump(data_pos, make_file, indent="\t")

# for i,name in enumerate(landry_name):
#     print(i)
#     query = name
#     print(query)
#     r = requests.get(url, params = {'query':query}, headers={'Authorization': 'KakaoAK '+ apikey})
#     js = simplejson.JSONEncoder().encode(r.json())
# #     print(r.json())

In [ ]:
new_data.head(10)

In [ ]:
df_landry = new_data.iloc[:, [3,4,6,11,12,13,14]]
list_landry = []

for j in range(len(data_landry)):
    list_landry.append(list(data_landry.loc[j]))
    
# print(list_landry)